In [43]:
import psycopg2
import numpy as np
import pandas as pd

In [52]:
# Connect to database
con = psycopg2.connect("dbname='SNZLab' user='postgres' host='localhost' password='PlantDevSchnLab112022'")

# Open a cursor to perform database operations
cur = con.cursor()

# Execute a command
cur.execute("SELECT * FROM cell_attributes")

In [53]:
# Obtain data as python opject

# Obtain first row of data table
# first_col = cur.fetchone()

# Obtain all rows
cell_attributes = cur.fetchall()

# Obtain column names
colnames = [desc[0] for desc in cur.description]

In [54]:
# Create pandas DataFrame from list of tuples
df = pd.DataFrame(cell_attributes)

# Add column names to respective columns
df.columns = colnames

In [55]:
df.head()

,cell_id,parent_label,genotype,sample_id,stage,cell_length_x,cell_length_y,cell_length_z,cell_wall_area,number_of_neighbors,outside_wall_area,outside_wall_area_ratio,volume
0,1835,7,ChiOX,1793,3-IV,6.00088,5.29523,6.09765,148.201,3,108.799,0.734131,142.288
1,1838,7,ChiOX,1793,3-IV,5.57317,4.97323,8.43972,147.736,7,105.308,0.712816,131.511
2,1842,7,ChiOX,1793,3-IV,7.01422,5.10685,9.65658,205.962,9,101.471,0.492667,228.485
3,1846,7,ChiOX,1793,3-IV,6.57697,4.84003,10.2757,221.655,13,114.535,0.516727,245.79
4,1847,7,ChiOX,1793,3-IV,5.87977,5.71539,6.19016,135.858,7,69.4824,0.511433,132.188
